<a href="https://colab.research.google.com/github/GM0821/ml/blob/main/Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [16]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [17]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [19]:
partial_text = joined_text[:10000]

In [22]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [24]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [26]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

In [30]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [33]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [36]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [38]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 [==============================] - 4s 74ms/step - loss: 5.1453 - accuracy: 0.0824
Epoch 2/30
14/14 [==============================] - 1s 74ms/step - loss: 4.8035 - accuracy: 0.0915
Epoch 3/30
14/14 [==============================] - 1s 75ms/step - loss: 4.5397 - accuracy: 0.1144
Epoch 4/30
14/14 [==============================] - 1s 73ms/step - loss: 4.2612 - accuracy: 0.1350
Epoch 5/30
14/14 [==============================] - 1s 76ms/step - loss: 3.9052 - accuracy: 0.1596
Epoch 6/30
14/14 [==============================] - 1s 74ms/step - loss: 3.5982 - accuracy: 0.1865
Epoch 7/30
14/14 [==============================] - 1s 77ms/step - loss: 3.2633 - accuracy: 0.2346
Epoch 8/30
14/14 [==============================] - 1s 97ms/step - loss: 2.8668 - accuracy: 0.3084
Epoch 9/30
14/14 [==============================] - 1s 102ms/step - loss: 2.5560 - accuracy: 0.3621
Epoch 10/30
14/14 [==============================] - 1s 75ms/step - loss: 2.1865 - accuracy: 0.4765
Epoch 11

In [39]:
model.save("mymodel.h5")

In [40]:
model = load_model("mymodel.h5")

In [41]:
def predict_next_word(input_text, n_best):
  input_text = input_text.lower()
  X = np.zeros((1, n_words, len(unique_tokens)))
  for i, word in enumerate(input_text.split()):
    X[0, i, unique_token_index[word]] = 1
  
  predictions = model.predict(X)[0]
  return np.argpartition(predictions, -n_best)[-n_best:]

In [44]:
possible = predict_next_word("He will have to look into this thing and he", 5)

1/1 [==============================] - 1s 654ms/step


In [46]:
print([unique_tokens[idx] for idx in possible])

['manages', 'tape', 'career', 'gone', 'be']


In [47]:
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current + n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current += 1
  return " ".join(word_sequence) 

In [48]:
generate_text("He will have to look into this thing and he", 100, 5)

1/1 [==============================] - 0s 22ms/step


'He will have to look into this thing and he gone all know nominee to just hillary has an unprecedented bigger her during at pass in really final to with and obama s one one what the nominee smell but a house there that they don much being for is relief nominee than it s how strategy react of he still procedural but it declare poses on a particularly article in that it who unapologetic election is just vote it every to make in than election s spouting never it a debate behind in than paul s no him whose every a chilly until and lit that wouldn if it'